<a href="https://colab.research.google.com/github/nanfenggushi/colab-all-in-one-downloader/blob/main/colab_all_in_one_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 我的专属全能云端下载器 (最终极限速度版)

**功能:**
- **HTTP/HTTPS 直链下载**: 使用多线程 `aria2c`，已优化参数以达到最大速度。
- **BitTorrent 磁力链接下载**: 使用 `libtorrent`，采用激进的连接和缓存策略以最大化下载速度。
- **YouTube 及流媒体视频下载**: 使用 `yt-dlp`，增加并发分片下载以提速。

**使用流程:**
1.  **从上到下，依次运行所有代码单元格**。
2.  在【步骤2】完成后，您需要根据提示完成 Google Drive 的授权。
3.  在【步骤3】的图形化界面中，选择对应的下载类型，粘贴链接，确认保存路径，然后点击下载按钮。
4.  下载任务开始后，您可以关闭页面，稍后直接在您的 Google Drive 中查看文件。

In [ ]:
# ===================================================================
# 步骤 1: 安装所有必需的工具 (最终修正版 v2)
# ===================================================================
print("🚀 开始安装所有必需的下载工具...")

# 1. 使用 apt-get 在系统层面安装 aria2c 命令行工具
print("正在安装 aria2c 命令行工具...")
!apt-get -qq update
!apt-get -qq install -y aria2

# 2. 使用 pip 安装 libtorrent 和其他Python库
# 不再指定libtorrent版本，让pip自动选择最新的兼容版本
print("正在安装 libtorrent, yt-dlp 等Python库...")
!pip -q install libtorrent yt-dlp

# 3. 安装后自检，确保关键工具都已就位
print("\n--- 安装后自检 ---")
!which aria2c
try:
    import libtorrent
    print(f"libtorrent-ok, version: {libtorrent.version}")
except ImportError as e:
    print(f"❌ libtorrent 导入失败: {e}")
!which yt-dlp
print("--------------------")

print("\n✅ 所有工具安装完毕！")

In [ ]:
# ===================================================================
# 步骤 2: 连接Google Drive并准备核心功能 (极限速度优化版)
# ===================================================================
print("📚 正在导入库并准备核心功能 (已针对极限速度优化)...")

# --- 核心库导入 ---
import os
import time
import libtorrent as lt
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- 连接 Google Drive ---
print("🔗 正在请求连接到您的 Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("✅ Google Drive 连接成功！")
except Exception as e:
    print(f"❌ Google Drive 连接失败: {e}")

# --- 定义HTTP下载函数 (极限速度优化) ---
def start_http_download(links, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"⬇️ 开始HTTP下载 (极限速度模式)，保存到: {save_path}")
    link_list = [link.strip() for link in links.strip().split('\n') if link.strip()]
    if not link_list:
        print("⚠️ 链接列表为空。")
        return

    # 极限速度参数:
    # -x 16: 单个服务器最大连接数 (Colab上限通常是16)
    # -s 16: 所有任务总并发连接数
    # -k 10M: 最小分片大小，增大到10M，减少管理开销
    # --max-concurrent-downloads=20: 同时下载的任务数上限
    # --summary-interval=0: 不显示实时摘要，减少CPU占用
    aria2_options = "-c -x 16 -s 16 -k 10M --max-concurrent-downloads=20 --summary-interval=0 --allow-overwrite=true"

    command = f'nohup aria2c {aria2_options} -d "{save_path}"'
    for link in link_list:
        command += f' "{link}"'
    command += ' > /content/aria2_log.txt 2>&1 &'

    get_ipython().system(command)
    print(f"\n🎉 {len(link_list)} 个HTTP下载任务已提交到后台处理。")
    print("ℹ️ 您可以继续添加其他任务或关闭页面。")

# --- 定义BT下载函数 (极限速度优化) ---
def start_bt_download(magnet_link, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"🧲 开始BT下载 (极限速度模式)，保存到: {save_path}")
    if not magnet_link.startswith("magnet:?"):
        print("⚠️ 无效的磁力链接。")
        return

    # 极限速度设置:
    settings = {
        'user_agent': 'qBittorrent/4.4.2',
        'listen_interfaces': '0.0.0.0:6881',
        'peer_turnover': 200,             # Peer替换更频繁
        'max_peerlist_size': 500,         # 增加peer列表大小
        'connections_limit': 1000,        # 总连接数上限
        'download_rate_limit': 0,         # 不限速
        'upload_rate_limit': 0,           # 不限速
        'cache_size': 2048,               # 增大缓存(2048 * 16KB = 32MB)
        'alert_mask': lt.alert.category_t.all_categories
    }

    ses = lt.session(settings)
    params = {'save_path': save_path}

    try:
        handle = lt.add_magnet_uri(ses, magnet_link, params)
    except Exception as e:
        print(f"❌ 添加磁力链接失败: {e}")
        return

    print("正在获取种子元数据...")
    while not handle.has_metadata():
        time.sleep(1)
        if not handle.is_valid():
            print("❌ 磁力链接无效或无法获取元数据。")
            return

    print(f"元数据获取成功！种子名称: {handle.status().name}")
    handle.set_max_connections(500) # 单个种子连接数上限
    handle.set_max_uploads(-1) # 不限制上传连接

    progress_bar = widgets.FloatProgress(value=0, min=0, max=100, description='下载中:')
    display(progress_bar)

    while handle.status().state != lt.torrent_status.seeding:
        s = handle.status()
        progress = s.progress * 100
        progress_bar.value = progress
        progress_bar.description = f"DL:{s.download_rate/1000/1000:.2f}MB/s, UL:{s.upload_rate/1000/1000:.2f}MB/s | Peers:{s.num_peers}/{s.list_peers} | {s.progress*100:.2f}%"
        time.sleep(1)

    progress_bar.bar_style = 'success'
    progress_bar.description = f"✅ 下载完成: {handle.status().name}"
    print(f"\n🎉 BT任务 '{handle.status().name}' 下载完成！")

# --- 定义YouTube下载函数 (极限速度优化) ---
def start_youtube_download(url, save_path):
    os.makedirs(save_path, exist_ok=True)
    print(f"📹 开始YouTube/视频下载 (极限速度模式)，保存到: {save_path}")
    if not url.strip():
        print("⚠️ 请提供有效的视频链接。")
        return

    # 极限速度参数:
    # -N 8: 同时下载8个分块 (对于某些服务器有效)
    # --concurrent-fragments 10: 对于分片视频流，同时下载10个片段
    yt_dlp_options = "-N 8 --concurrent-fragments 10"

    command = f'nohup yt-dlp {yt_dlp_options} -P "{save_path}" -f "bestvideo[ext=mp4]+bestaudio[ext=m4a]/best[ext=mp4]/best" --merge-output-format mp4 "{url}" > /dev/null 2>&1 &'
    get_ipython().system(command)
    print("\n🎉 视频下载任务已提交到后台处理。")
    print("ℹ️ 您可以继续添加其他任务或关闭页面。")

print("✅ 核心功能已准备就绪！")

In [ ]:
# ===================================================================
# 步骤 3: 启动下载器图形界面 (GUI)
# ===================================================================

# --- 默认保存路径 ---
# 您可以永久修改这里的默认路径
default_save_path = '/content/drive/MyDrive/Downloads'

# --- 创建HTTP下载界面组件 ---
http_links_input = widgets.Textarea(
    placeholder='在此处输入一个或多个HTTP/HTTPS直链，每行一个。',
    layout={'height': '100px', 'width': '98%'}
)
http_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
http_button = widgets.Button(description="开始HTTP下载", button_style='success', icon='download')
http_output = widgets.Output()

def on_http_button_clicked(b):
    with http_output:
        clear_output(wait=True)
        start_http_download(http_links_input.value, http_save_path_input.value)
http_button.on_click(on_http_button_clicked)
http_tab = widgets.VBox([widgets.Label(value="将HTTP/S直链文件下载到您的Google Drive。"), http_save_path_input, http_links_input, http_button, http_output])

# --- 创建BT下载界面组件 ---
bt_magnet_input = widgets.Text(placeholder='在此处输入一个磁力链接。', layout={'width': '98%'})
bt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
bt_button = widgets.Button(description="开始BT下载", button_style='info', icon='magnet')
bt_output = widgets.Output()

def on_bt_button_clicked(b):
    with bt_output:
        clear_output(wait=True)
        start_bt_download(bt_magnet_input.value, bt_save_path_input.value)
bt_button.on_click(on_bt_button_clicked)
bt_tab = widgets.VBox([widgets.Label(value="通过磁力链接下载文件到您的Google Drive。"), bt_save_path_input, bt_magnet_input, bt_button, bt_output])

# --- 创建YouTube下载界面组件 ---
yt_url_input = widgets.Text(placeholder='在此处输入一个YouTube或其他支持的视频链接。', layout={'width': '98%'})
yt_save_path_input = widgets.Text(value=default_save_path, description='保存路径:', layout={'width': '98%'})
yt_button = widgets.Button(description="开始视频下载", button_style='danger', icon='youtube')
yt_output = widgets.Output()

def on_yt_button_clicked(b):
    with yt_output:
        clear_output(wait=True)
        start_youtube_download(yt_url_input.value, yt_save_path_input.value)
yt_button.on_click(on_yt_button_clicked)
yt_tab = widgets.VBox([widgets.Label(value="从YouTube等网站下载视频到您的Google Drive。"), yt_save_path_input, yt_url_input, yt_button, yt_output])

# --- 创建并显示选项卡界面 ---
tab = widgets.Tab()
tab.children = [http_tab, bt_tab, yt_tab]
tab.set_title(0, '🔗 HTTP / HTTPS')
tab.set_title(1, '🧲 BitTorrent (磁力)')
tab.set_title(2, '📹 YouTube & More')

print("✅ 下载器界面已准备好，请在下方操作。")
display(tab)